In [1]:
import requests
import pandas as pd
import time
import talib
import pytz
import datetime
import os
import ibapi
from ib_insync import *

In [2]:
util.startLoop()  # uncomment this line when in a notebook
ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)

In [3]:
API_KEY = 'P9DHEUMVT6580QQX'
INTERVAL = '1min'

In [4]:
# Определение Ticker и Кол-ва акций для торговли
ticker = str(input('Enter ticker: ')).upper() 
count = int(input('Amount: ')) 

Enter ticker: SPY
Amount: 10


In [5]:
if not ib.isConnected():  # connect only without active connection    
    try:
        print('Connecting...')
        ib.connect('127.0.0.1', 7497, clientId=1)  # connect
    except Exception as ex:
        print('Error:', ex)  # catch at exception

if ib.isConnected():
    print("Successfully connected")

def balance():
    balances = {av.tag: float(av.value) for av in ib.accountSummary()
                if av.tag in ['AvailableFunds', 'BuyingPower', 'TotalCashValue', 'NetLiquidation']}
    balance = balances.get('AvailableFunds', 0)
    
    return balance

def check_balance(ticker, count):
    price = read_data(ticker)['close'].iloc[-1]
    amount = price * count
    if balance() > amount:
        return True
    else:
        return False

'''
==============================================================================
                       Функция проверки работы биржи
      Определяет состояние: premarket, regular, postmarket или close
==============================================================================
'''
def time_check(ticker):
    contract = Stock(ticker)
    cds = ib.reqContractDetails(contract)
    hours = cds[0].tradingHours
    hourslist1 = hours.split(';')
    hourslist2 = hourslist1[0].split('-')
    hourslistopening = hourslist2[0].split(':')
    tz = get_tz(ticker)
    today = datetime.datetime.now(tz=pytz.UTC).astimezone(pytz.timezone(tz))
    date = today.strftime("%Y%m%d")
    time = today.strftime("%H%M")
    print('DateTime: ', today.strftime("%d-%m-%Y at %I:%M%p"), tz)
    
    if hourslistopening[1] == 'CLOSED':
        return 'close' 
    else:
        hourslistclosing = hourslist2[1].split(':')
        openingsdict = dict(zip(hourslistopening[::2], hourslistopening[1::2]))
        closingdict = dict(zip(hourslistclosing[::2], hourslistclosing[1::2]))
        hoursregular = cds[0].liquidHours
        hourslist1regular = hoursregular.split(';')
        hourslist2regular = hourslist1regular[0].split('-')
        hourslistopeningregular = hourslist2regular[0].split(':')
        hourslistclosingregular = hourslist2regular[1].split(':')
        openingsdictregular = dict(zip(hourslistopeningregular[::2], hourslistopeningregular[1::2]))
        closingdictregular = dict(zip(hourslistclosingregular[::2], hourslistclosingregular[1::2]))

        rangelist = []

        for key, value in openingsdict.items():
            rangelist.append(value)
    
        for key, value in closingdict.items():
            rangelist.append(value)

        for key, value in openingsdictregular.items():
            rangelist.append(value)
    
        for key, value in closingdictregular.items():
            rangelist.append(value)    
       
        sortrangelist = sorted(rangelist) 

        if sortrangelist[0] <= time < sortrangelist[1]:
            return 'premarket' 
        elif sortrangelist[1] <= time < sortrangelist[2]:
            return 'regular' 
        elif sortrangelist[2] <= time < sortrangelist[3]:
            return 'postmarket'
        else:
            return 'close'
    
# def get_tz(ticker):
#     filename = f'{ticker}.json'
#     url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={INTERVAL}&apikey={API_KEY}'
#     r = requests.get(url)
#     content = r.content.decode('UTF-8')
#     with open(filename, "w") as file:
#         file.write(content)
#     data = pd.read_json(f'{ticker}.json')
#     tz = data['Meta Data']['6. Time Zone']   
#     os.remove(f'{ticker}.json')
    
#     return tz

'''
==============================================================================
                    Функция получения часового пояса ticker'а
==============================================================================
'''
def get_tz(ticker):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={INTERVAL}&apikey={API_KEY}'
    r = requests.get(url)
    json_data = r.json()
    tz = json_data['Meta Data']['6. Time Zone']
    
    return tz    

'''
==============================================================================
                    Функция загрузки данных в файл {ticker}.csv
==============================================================================
'''
def load_data(ticker):
    filename = f'{ticker}.csv'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={INTERVAL}&apikey={API_KEY}&datatype=csv&outputsize=compact'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(filename, "w") as file:
        file.write(content)
    print(f'\tUpdate {ticker} price history:', filename)
    
'''
==============================================================================
                        Функция чтения биржевых данных
==============================================================================
'''
def read_data(ticker):
    data = pd.read_csv(f'{ticker}.csv')
    os.remove(f'{ticker}.csv')
        
    return data

def algorithm(ticker):
    try:
        close = read_data(ticker)['close'].fillna(method='ffill')
    except KeyError:
        print('No data, the limit of requests may be exceeded')
        disconnect()
    else:
        ma_long = talib.SMA(close, timeperiod=26)
        ma_short = talib.SMA(close, timeperiod=9)

        last_price = close.iloc[-1]

        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-1]:.4f} Long={ma_long.iloc[-1]:.4f}')
        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-2]:.4f} Long={ma_long.iloc[-2]:.4f} (previous)')

        if ma_long.iloc[-2] > ma_short.iloc[-2] and ma_long.iloc[-1] < ma_short.iloc[-1]:
                contract = get_contract(ticker)
                buy(ticker, contract)

        elif ma_long.iloc[-2] < ma_short.iloc[-2] and ma_long.iloc[-1] > ma_short.iloc[-1]:
                contract = get_contract(ticker)
                sell(ticker, contract)
        
def get_contract(ticker):
    contract = Stock(f'{ticker}', 'SMART', 'USD')
    ib.qualifyContracts(contract)
    
    return contract
    
def list_positions():
    positions = ib.positions()
    positions = "\n".join([f"{p.contract.localSymbol} {p.position}x{p.avgCost}"
                               for p in positions])
    return positions

def list_orders():
    trades = ib.openTrades()
    orders = "\n".join([f"{t.order.action} {t.contract.secType} {t.contract.symbol} {t.contract.localSymbol}"
                            f" {t.order.totalQuantity}x{t.order.lmtPrice}"
                            for t in trades])
    return orders

def buy(ticker, contract): 
    if f'{ticker}' not in list_positions():
        if f'{ticker}' not in list_orders():
            order = MarketOrder('BUY', f'{count}') 
            ib.placeOrder(contract, order)
        
def sell(ticker, contract):
    if f'{ticker}' in list_positions():
         if f'{ticker}' not in list_orders():
            order = MarketOrder('SELL',  f'{count}')
            ib.placeOrder(contract, order)
            
def run_algorithm():
    messages = {
      'premarket': 'Premarket',
      'postmarket': 'Postmarket',
      'close': 'Closed',
      'regular': 'Regular session',
    } 
    message = messages.get(time_check(ticker), 'Uncertain')
    print(message)
        
    if message == 'Regular session':
        load_data(ticker) 
        if check_balance(ticker, count):        
            print('== Start working ==')

            while True:

                load_data(ticker)
                algorithm(ticker)
                time.sleep(60)
                
        else:
            print('Insufficient funds')
    else:
        print('Wait for the regular session to open')

def disconnect():
    if ib.isConnected():  # check for connection
        ib.disconnect()  # disconnect

        while ib.isConnected():  # wait while disconnecting
            time.sleep(1)  # sleep 1 sec on waiting

    print("Successful disconnected with TWS")

Connecting...
Successfully connected


In [6]:
if ib.isConnected():
    try:  
        run_algorithm()
    except KeyboardInterrupt:
        disconnect()
        print('== Stop working ==')
else:
    print('Error: Not connected')

DateTime:  17-02-2020 at 05:11AM US/Eastern
Closed
Wait for the regular session to open


In [6]:
disconnect()

Successful disconnected with TWS


In [7]:
get_contract(ticker)

Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')

In [26]:
BASE_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={INTERVAL}&apikey={API_KEY}'

r = requests.get(BASE_URL)
json_data = r.json()

# print(parsed_json)

name = json_data['Meta Data']['6. Time Zone']

print(name)

US/Eastern


In [8]:
'''
============================================================================
            Функция загрузки данных через json
============================================================================           
'''
def my_load_data(ticker):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={INTERVAL}&apikey={API_KEY}&datatype=json&outputsize=compact'
    r = requests.get(url)
    json_data = r.json()
    
    #Определить название ключа, оно зависит от выбранного INTERVAL
    key_name = list( json_data.keys() )[1]
    
    # Загрузить данные. Важно! нужно заменить ' на ", с ' не работает.
    data = pd.read_json(str(json_data[key_name]).replace('\'', '\"'), orient='index')
    
    # Привести данные к формату файла *.csv
    data = data.reset_index() \
               .rename(columns={'index'    : 'timestamp',
                                '1. open'  : 'open',
                                '2. high'  : 'high',
                                '3. low'   : 'low',
                                '4. close' : 'close',
                                '5. volume': 'vloume'})
    return data

In [9]:
stock_data = my_load_data(ticker)
stock_data

,timestamp,open,high,low,close,vloume
0,2020-02-14 16:00:00,337.580,337.630,337.550,337.600,1005288
1,2020-02-14 15:59:00,337.535,337.600,337.530,337.590,339257
2,2020-02-14 15:58:00,337.550,337.590,337.525,337.540,252776
3,2020-02-14 15:57:00,337.480,337.560,337.475,337.560,259297
4,2020-02-14 15:56:00,337.460,337.515,337.400,337.475,293746
...,...,...,...,...,...,...
95,2020-02-14 14:25:00,336.670,336.690,336.640,336.680,69129
96,2020-02-14 14:24:00,336.525,336.680,336.525,336.680,73090
97,2020-02-14 14:23:00,336.495,336.530,336.460,336.530,20540
98,2020-02-14 14:22:00,336.455,336.510,336.455,336.500,47842


In [45]:
# stock_data['close'].iloc[-1]
balance()

1272758.51

In [48]:
print(list_positions())

ESGV 1142.0x52.7765556
ESGD 954.0x64.1715282
DVY 292.0x101.06176745
VO 50.0x170.48881
VDE 99.0x81.962902
TFI 1109.0x50.50644005
IBKR 2800.0x39.4058383
TIP 207.0x115.00465265
BWX 712.0x28.57725195
LQD 99.0x124.4389121
ESML 309.0x27.38696345
VNQ 137.0x88.7322321
ESGE 1221.0x33.994677
EAGG 1280.0x53.3022901


In [ ]:
def list_positions():
    positions = ib.positions()
    positions = "\n".join([f"{p.contract.localSymbol} {p.position}x{p.avgCost}"
                               for p in positions])
    return positions

In [79]:
positions = ib.positions()
# [str(p.contract.localSymbol) + ' - ' + str(p.position) + 'x' + str(p.avgCost) for p in positions]
positions[0]

Position(account='DUC00074', contract=Stock(conId=333970434, symbol='ESGV', exchange='BATS', currency='USD', localSymbol='ESGV', tradingClass='ESGV'), position=1142.0, avgCost=52.7765556)